In [5]:
import ee
import geemap
import numpy as np
import sys
import geopandas as gpd
sys.path.insert(1, '/Users/gopal/Projects/ml/downloadGEErasters')
sys.path.insert(1, '/Users/gopal/Projects/sasia/src/proc/')
import rs
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
# from rs import prep_tm5_ic, prep_etm7_ic, prep_oli8_ic, prep_landsat_clouds

In [6]:
ee.Initialize()

In [7]:
ee_geometry = ee.FeatureCollection('users/gopalpenny/arkavathytanks/ArkavatiSubbasins')
tg_halli = ee_geometry.filter(ee.Filter.eq('Subcatch', 'TG_Halli'))
tg_halli.aggregate_array('Subcatch').getInfo()

['TG_Halli']

In [8]:

tm5 = rs.prep_tm5_ic(ee_geometry).filterDate('2014-06-01','2015-06-01')
etm7 = rs.prep_etm7_ic(ee_geometry).filterDate('2014-06-01','2015-06-01')
oli8 = rs.prep_oli8_ic(tg_halli).filterDate('2014-06-01','2015-06-01')

landsat = tm5.merge(etm7).merge(oli8) \
    .filter(ee.Filter.lt('CLOUD_COVER', 50))

landsat_clouds = rs.prep_landsat_clouds(landsat)

In [9]:
# landsat = landsat #\
  # .filterDate('2014-11-01','2015-02-01') #\
  # .filter(ee.Filter.inList('DATE_ACQUIRED', ee.List(['2014-11-10','2015-01-13'])))

landsat_clouds.aggregate_array('DATE_ACQUIRED').getInfo()

['2014-06-11',
 '2014-06-27',
 '2014-08-14',
 '2014-09-15',
 '2014-10-01',
 '2014-10-17',
 '2014-11-18',
 '2014-12-04',
 '2014-12-20',
 '2015-01-05',
 '2015-01-21',
 '2015-02-06',
 '2015-02-22',
 '2015-03-10',
 '2015-04-27',
 '2015-05-29',
 '2014-06-11',
 '2014-06-27',
 '2014-08-14',
 '2014-09-15',
 '2014-10-01',
 '2014-11-18',
 '2014-12-04',
 '2015-01-05',
 '2015-01-21',
 '2015-02-06',
 '2015-02-22',
 '2015-03-10',
 '2015-04-11',
 '2015-04-27',
 '2014-08-22',
 '2014-09-23',
 '2014-10-09',
 '2014-11-10',
 '2014-11-26',
 '2015-01-13',
 '2015-01-29',
 '2015-02-14',
 '2015-03-18',
 '2015-04-03',
 '2015-04-19',
 '2015-05-05',
 '2015-05-21']

In [30]:

samp_pts_full = gpd.read_file('../../spatial/unmod/training/Landuse_village_datapoints_15_01_2015.shp')
samp_pts = samp_pts_full[['Unique_ID','Longitude','Latitude','geometry']]
samp_pts


,Unique_ID,Longitude,Latitude,geometry
0,1_MAPIN_13/11/2014,77.3006,13.1950,POINT (77.30060 13.19500)
1,2_MAPIN_13/11/2014,77.3016,13.1953,POINT (77.30160 13.19530)
2,3_MAPIN_13/11/2014,77.3025,13.1958,POINT (77.30250 13.19580)
3,4_MAPIN_13/11/2014,77.3032,13.1959,POINT (77.30320 13.19590)
4,5_MAPIN_13/11/2014,77.3033,13.1953,POINT (77.30330 13.19530)
...,...,...,...,...
614,1809_GPS_22/08/2014,77.2738,13.2742,POINT (77.27383 13.27420)
615,1810_GPS_22/08/2014,77.2720,13.2635,POINT (77.27198 13.26347)
616,1811_GPS_22/08/2014,77.2698,13.2452,POINT (77.26981 13.24523)
617,1812_GPS_22/08/2014,77.2850,13.2231,POINT (77.28504 13.22306)


In [31]:
samp_pts_ee = geemap.geopandas_to_ee(samp_pts)

# oli8.select([0]).projection()



In [33]:
# print(rs.get_pixel_ts_allbands.__doc__)
training_ts = rs.get_pixel_ts_allbands(samp_pts_ee, landsat_clouds, 'DATE_ACQUIRED', scale = 30)
# import inspect
# print(inspect.getsource(rs.get_pixel_ts_allbands))

# crops_task = ee.batch.Export.image.toDrive(
#     image = crops_bands,
#     description = 'crops_bands',
#     folder = 'classy_downloads',
#     fileNamePrefix = 'karur_crops_2015_2021',
#     region = ee_geometry_box,
#     scale = 30)

# # crops_task.start()
export_ts = ee.batch.Export.table.toDrive(
    collection = training_ts,
    description = 'training_ts',
    folder = 'gee_sasia',
    fileNamePrefix = 'landsat_for_lulc_2014_2015',
    fileFormat = 'CSV')

export_ts.start()

In [34]:
ee.batch.Task.list()

[<Task 2VX2HDDDGHADYNXN63F2BYSL EXPORT_FEATURES: training_ts (COMPLETED)>,
 <Task 6JV7R35NSF6CDMQLBBOX6OJW EXPORT_IMAGE: crops_bands (COMPLETED)>]

In [14]:
ts_gpd = geemap.ee_to_geopandas(ts)
ts_gpd.to_file('../../data/format/get_landsat_for_lulc/lulc_oli8.shp')

Exception: Collection query aborted after accumulating over 5000 elements.

In [74]:
m = geemap.Map()
m.addLayer(oli8.filterDate('2014-11-01','2014-12-01').first(),{'bands':['nir','red','green'], 'max': 30000},'oli8 2014')
m.addLayer(oli8.filterDate('2015-01-01','2015-12-01').first(),{'bands':['nir','red','green'], 'max': 30000},'oli8 2015')
m.addLayer(ee.Image().paint(ee_geometry,1,1),{'palette':'white'},'ark')
m.addLayerControl()
m.centerObject(ee_geometry, 10)
m

Map(center=[12.84852761341192, 77.44092789490166], controls=(WidgetControl(options=['position', 'transparent_b…